<a href="https://colab.research.google.com/github/maawoo/nithecs_microschool_xarray_eo/blob/main/01_intro_to_xarray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

this is a test

In [1]:
import xarray as xr

In [2]:
print(xr.__version__)

2025.1.2
